### Import Libraries

In [7]:
import pandas as pd
import re

# NLTK
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

# NLTK Resources
nltk.download("punkt") # for word_tokenize
nltk.download("stopwords") # for stopwords
nltk.download("wordnet") # for WordNetLemmatizer
nltk.download("omw-1.4") # for WordNetLemmatizer to work with multiple languages
nltk.download("averaged_perceptron_tagger") # for nltk.pos_tag

# plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report

# TensorFlow/Keras
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.models import Model



[nltk_data] Downloading package punkt to /Users/meng/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/meng/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/meng/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/meng/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/meng/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### Load Dataset

In [2]:
data = pd.read_csv('news_dataset.csv')

In [3]:
print(data.head())

                                               title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date  label  
0  December 31, 2017      0  
1  December 31, 2017      0  
2  December 30, 2017      0  
3  December 29, 2017      0  
4  December 25, 2017      0  


### EDA

In [4]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38729 entries, 0 to 38728
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    38729 non-null  object
 1   text     38729 non-null  object
 2   subject  38729 non-null  object
 3   date     38729 non-null  object
 4   label    38729 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.5+ MB
None


In [5]:
print(data.isnull().sum())

title      0
text       0
subject    0
date       0
label      0
dtype: int64


In [6]:
print(data['label'].value_counts())

label
1    20826
0    17903
Name: count, dtype: int64


In [8]:
# plot the balance of the dataset
plt.figure(figsize=(8, 5))
sns.countplot(x='label', data=data)
plt.title('Distribution of Labels')
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()

In [10]:
# longest sentence length 
def longest_sentence_length(text):
  return len(text.split())

data['maximum_length'] = data['title'].apply(lambda x : longest_sentence_length(x))
print('longest sentence having length:')
max_length = max(data['maximum_length'].values)
print(max_length)

longest sentence having length:
42


### Pre-processing

In [11]:
# Initialisation
stop_words_list = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

# POS tag conversion function: Change 'NLTK.pos_tag' Pos tag format to WordNet format
def convert_to_wordnet_pos(tag):
    if tag.startswith("J"):
        return wordnet.ADJ
    
    elif tag.startswith("V"):
        return wordnet.VERB
    
    elif tag.startswith("N"):
        return wordnet.NOUN
    
    elif tag.startswith("R"):
        return wordnet.ADV
    
    else:
        return wordnet.NOUN

# Text Preprocessing function
def preprocess_text(text):
    # Lowercase
    text = text.lower()
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # Remove special characters
    text = re.sub(r"[^\w\s']", '', text)  # Keep ' symbokl for contractions

    # Remove Mentions
    text = re.sub(r'@\w+', '', text)

    # Remove Hashtags
    text = re.sub(r'#\w+', '', text)

    # Remove extra spaces and newlines
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Tokenization
    tokens_list = word_tokenize(text)
    
    # Remove stop words
    filtered_tokens = [word for word in tokens_list if word not in stop_words_list]
    
    # POS tagging
    # nltk.pos_tag will return a list of tuples (word, tag)
    tagged_tokens = nltk.pos_tag(filtered_tokens)
    
    # Lemmatization with POS tagging
    lemmatized_tokens = [
        lemmatizer.lemmatize(word, pos = convert_to_wordnet_pos(tag))
        for word, tag in tagged_tokens
    ]
    
    return " ".join(lemmatized_tokens)

In [12]:
data['cleaned_text'] = data['text'].apply(preprocess_text)

In [13]:
data['cleaned_title'] = data['title'].apply(preprocess_text)

In [10]:
print(data['cleaned_text'].head())
print(data['cleaned_title'].head())

0    donald trump wish american happy new year leav...
1    house intelligence committee chairman devin nu...
2    friday reveal former milwaukee sheriff david c...
3    christmas day donald trump announce would back...
4    pope francis use annual christmas day message ...
Name: cleaned_text, dtype: object
0    donald trump sends embarrass new year eve mess...
1    drunk brag trump staffer start russian collusi...
2    sheriff david clarke become internet joke thre...
3      trump obsess even obamas name cod website image
4      pope francis call donald trump christmas speech
Name: cleaned_title, dtype: object


### Split Dataset

In [44]:
X_text = data["cleaned_text"]
y_text = data["label"]

X_train_text, X_test_text, y_train_text, y_test_text = train_test_split(
    X_text, y_text, test_size = 0.2, random_state = 42, stratify = y_text
)

In [45]:
X_title = data["cleaned_title"]
y_title = data["label"]

X_train_title, X_test_title, y_train_title, y_test_title = train_test_split(
    X_title, y_title, test_size = 0.2, random_state = 42, stratify = y_title
)

### Train Models

#### 0. Pipeline Function

In [46]:
def pipeline(
    model_type,
    vectorizer_type,
    X_train,
    y_train,
    X_test,
    y_test,
    ngram_range):

    # Selecting the vectorizer
    if vectorizer_type == 'tfidf':
        vectorizer = TfidfVectorizer(max_features = 250, ngram_range = ngram_range)

    elif vectorizer_type == 'bow':
        vectorizer = CountVectorizer(
            max_features = 250,   # limit the number of features to 250
            min_df = 10,           # at least 10 documents must contain the word
            max_df = 0.6,          # at most 60% of documents can contain the word
            ngram_range = ngram_range # (1, 2) # 1-gram and 2-gram
        )

    else:
        raise ValueError("vectorizer_type must be either 'tfidf' or 'bow'")
    
    # Selecting the model
    if model_type == 'logistic':
        model = LogisticRegression(max_iter = 1000, random_state = 42)
    
    elif model_type == 'svm':
        model = SVC(random_state = 42)
    
    elif model_type == 'naive_bayes':
        model = MultinomialNB()
    
    elif model_type == 'random_forest':
        model = RandomForestClassifier(random_state = 42)
    
    else:
        raise ValueError("model_type must be either 'logistic', 'svm', 'naive_bayes', or 'random_forest'")

    # Creating the pipeline
    pipe = Pipeline([
        ("vectorizer", vectorizer),
        ("classifier", model)
    ])

    # Fitting the model
    pipe.fit(X_train, y_train)

    # Making predictions
    y_pred_test = pipe.predict(X_test)

    # Evaluating the model
    accuracy = accuracy_score(y_test, y_pred_test)
    report = classification_report(y_test, y_pred_test, output_dict = False)

    # Getting the model name
    modelname = f"{type(model).__name__} with {type(vectorizer).__name__}"

    # Printing the results
    print(f"===== Model: {modelname} =====")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Classification Report:\n{report}")

    return pipe

#### BoW + Logistic Regression

In [46]:
pipeline(
    model_type = 'logistic',
    vectorizer_type = 'bow',
    X_train = X_train_text,
    y_train = y_train_text,
    X_test = X_test_text,
    y_test = y_test_text,
    ngram_range = (1, 1)
)

===== Model: LogisticRegression with CountVectorizer =====
Accuracy: 0.9904
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      3581
           1       0.99      0.99      0.99      4165

    accuracy                           0.99      7746
   macro avg       0.99      0.99      0.99      7746
weighted avg       0.99      0.99      0.99      7746



Pipeline(steps=[('vectorizer',
                 CountVectorizer(max_df=0.6, max_features=500, min_df=10)),
                ('classifier',
                 LogisticRegression(max_iter=1000, random_state=42))])

In [49]:
pipeline(
    model_type = 'logistic',
    vectorizer_type = 'bow',
    X_train = X_train_title,
    y_train = y_train_title,
    X_test = X_test_title,
    y_test = y_test_title,
    ngram_range = (1, 1)
)

===== Model: LogisticRegression with CountVectorizer =====
Accuracy: 0.8705
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.81      0.85      3581
           1       0.85      0.93      0.88      4165

    accuracy                           0.87      7746
   macro avg       0.88      0.87      0.87      7746
weighted avg       0.87      0.87      0.87      7746



Pipeline(steps=[('vectorizer',
                 CountVectorizer(max_df=0.6, max_features=250, min_df=10)),
                ('classifier',
                 LogisticRegression(max_iter=1000, random_state=42))])

### Bag of Words

#### BoW + Naive Bayes

In [50]:
pipeline(
    model_type = 'naive_bayes',
    vectorizer_type = 'bow',
    X_train = X_train_text,
    y_train = y_train_text,
    X_test = X_test_text,
    y_test = y_test_text,
    ngram_range = (1, 1)
)

===== Model: MultinomialNB with CountVectorizer =====
Accuracy: 0.9141
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.87      0.90      3581
           1       0.90      0.95      0.92      4165

    accuracy                           0.91      7746
   macro avg       0.92      0.91      0.91      7746
weighted avg       0.92      0.91      0.91      7746



Pipeline(steps=[('vectorizer',
                 CountVectorizer(max_df=0.6, max_features=250, min_df=10)),
                ('classifier', MultinomialNB())])

In [51]:
pipeline(
    model_type = 'naive_bayes',
    vectorizer_type = 'bow',
    X_train = X_train_title,
    y_train = y_train_title,
    X_test = X_test_title,
    y_test = y_test_title,
    ngram_range = (1, 1)
)

===== Model: MultinomialNB with CountVectorizer =====
Accuracy: 0.8570
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.84      0.84      3581
           1       0.86      0.87      0.87      4165

    accuracy                           0.86      7746
   macro avg       0.86      0.86      0.86      7746
weighted avg       0.86      0.86      0.86      7746



Pipeline(steps=[('vectorizer',
                 CountVectorizer(max_df=0.6, max_features=250, min_df=10)),
                ('classifier', MultinomialNB())])

#### Bow + SVM

In [47]:
pipeline(
    model_type = 'svm',
    vectorizer_type = 'bow',
    X_train = X_train_text,
    y_train = y_train_text,
    X_test = X_test_text,
    y_test = y_test_text,
    ngram_range = (1, 1)
)

===== Model: SVC with CountVectorizer =====
Accuracy: 0.9901
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      3581
           1       0.99      0.99      0.99      4165

    accuracy                           0.99      7746
   macro avg       0.99      0.99      0.99      7746
weighted avg       0.99      0.99      0.99      7746



Pipeline(steps=[('vectorizer',
                 CountVectorizer(max_df=0.6, max_features=250, min_df=10)),
                ('classifier', SVC(random_state=42))])

In [48]:
pipeline(
    model_type = 'svm',
    vectorizer_type = 'bow',
    X_train = X_train_title,
    y_train = y_train_title,
    X_test = X_test_title,
    y_test = y_test_title,
    ngram_range = (1, 1)
)

===== Model: SVC with CountVectorizer =====
Accuracy: 0.8730
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.80      0.85      3581
           1       0.85      0.93      0.89      4165

    accuracy                           0.87      7746
   macro avg       0.88      0.87      0.87      7746
weighted avg       0.88      0.87      0.87      7746



Pipeline(steps=[('vectorizer',
                 CountVectorizer(max_df=0.6, max_features=250, min_df=10)),
                ('classifier', SVC(random_state=42))])

#### Bow + Random Forest

In [49]:
pipeline(
    model_type = 'random_forest',
    vectorizer_type = 'bow',
    X_train = X_train_text,
    y_train = y_train_text,
    X_test = X_test_text,
    y_test = y_test_text,
    ngram_range = (1, 1)
)

===== Model: RandomForestClassifier with CountVectorizer =====
Accuracy: 0.9946
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      3581
           1       0.99      1.00      0.99      4165

    accuracy                           0.99      7746
   macro avg       0.99      0.99      0.99      7746
weighted avg       0.99      0.99      0.99      7746



Pipeline(steps=[('vectorizer',
                 CountVectorizer(max_df=0.6, max_features=250, min_df=10)),
                ('classifier', RandomForestClassifier(random_state=42))])

In [50]:
pipeline(
    model_type = 'random_forest',
    vectorizer_type = 'bow',
    X_train = X_train_title,
    y_train = y_train_title,
    X_test = X_test_title,
    y_test = y_test_title,
    ngram_range = (1, 1)
)

===== Model: RandomForestClassifier with CountVectorizer =====
Accuracy: 0.8617
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.82      0.85      3581
           1       0.85      0.90      0.87      4165

    accuracy                           0.86      7746
   macro avg       0.86      0.86      0.86      7746
weighted avg       0.86      0.86      0.86      7746



Pipeline(steps=[('vectorizer',
                 CountVectorizer(max_df=0.6, max_features=250, min_df=10)),
                ('classifier', RandomForestClassifier(random_state=42))])

### TF-IDF

#### TF-IDF + Logistic Regression

In [51]:
pipeline(
    model_type = 'logistic',
    vectorizer_type = 'tfidf',
    X_train = X_train_text,
    y_train = y_train_text,
    X_test = X_test_text,
    y_test = y_test_text,
    ngram_range = (1, 1)
)

===== Model: LogisticRegression with TfidfVectorizer =====
Accuracy: 0.9779
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.97      0.98      3581
           1       0.97      0.98      0.98      4165

    accuracy                           0.98      7746
   macro avg       0.98      0.98      0.98      7746
weighted avg       0.98      0.98      0.98      7746



Pipeline(steps=[('vectorizer', TfidfVectorizer(max_features=250)),
                ('classifier',
                 LogisticRegression(max_iter=1000, random_state=42))])

In [52]:
pipeline(
    model_type = 'logistic',
    vectorizer_type = 'tfidf',
    X_train = X_train_title,
    y_train = y_train_title,
    X_test = X_test_title,
    y_test = y_test_title,
    ngram_range = (1, 1)
)

===== Model: LogisticRegression with TfidfVectorizer =====
Accuracy: 0.8708
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.82      0.85      3581
           1       0.85      0.92      0.88      4165

    accuracy                           0.87      7746
   macro avg       0.87      0.87      0.87      7746
weighted avg       0.87      0.87      0.87      7746



Pipeline(steps=[('vectorizer', TfidfVectorizer(max_features=250)),
                ('classifier',
                 LogisticRegression(max_iter=1000, random_state=42))])

#### TF-IDF + Naive Bayes

In [53]:
pipeline(
    model_type = 'naive_bayes',
    vectorizer_type = 'tfidf',
    X_train = X_train_text,
    y_train = y_train_text,
    X_test = X_test_text,
    y_test = y_test_text,
    ngram_range = (1, 1)
)

===== Model: MultinomialNB with TfidfVectorizer =====
Accuracy: 0.9060
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.86      0.89      3581
           1       0.88      0.95      0.92      4165

    accuracy                           0.91      7746
   macro avg       0.91      0.90      0.90      7746
weighted avg       0.91      0.91      0.91      7746



Pipeline(steps=[('vectorizer', TfidfVectorizer(max_features=250)),
                ('classifier', MultinomialNB())])

In [54]:
pipeline(
    model_type = 'naive_bayes',
    vectorizer_type = 'tfidf',
    X_train = X_train_title,
    y_train = y_train_title,
    X_test = X_test_title,
    y_test = y_test_title,
    ngram_range = (1, 1)
)

===== Model: MultinomialNB with TfidfVectorizer =====
Accuracy: 0.8573
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.83      0.84      3581
           1       0.86      0.88      0.87      4165

    accuracy                           0.86      7746
   macro avg       0.86      0.86      0.86      7746
weighted avg       0.86      0.86      0.86      7746



Pipeline(steps=[('vectorizer', TfidfVectorizer(max_features=250)),
                ('classifier', MultinomialNB())])

#### TF-IDF + SVM

In [14]:
pipeline(
    model_type = 'svm',
    vectorizer_type = 'tfidf',
    X_train = X_train_text,
    y_train = y_train_text,
    X_test = X_test_text,
    y_test = y_test_text,
    ngram_range = (1, 1)
)

===== Model: SVC with TfidfVectorizer =====
Accuracy: 0.9901
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      3581
           1       0.99      0.99      0.99      4165

    accuracy                           0.99      7746
   macro avg       0.99      0.99      0.99      7746
weighted avg       0.99      0.99      0.99      7746



Pipeline(steps=[('vectorizer', TfidfVectorizer(max_features=250)),
                ('classifier', SVC(random_state=42))])

In [15]:
pipeline(
    model_type = 'svm',
    vectorizer_type = 'tfidf',
    X_train = X_train_title,
    y_train = y_train_title,
    X_test = X_test_title,
    y_test = y_test_title,
    ngram_range = (1, 1)
)

===== Model: SVC with TfidfVectorizer =====
Accuracy: 0.8739
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.81      0.86      3581
           1       0.85      0.93      0.89      4165

    accuracy                           0.87      7746
   macro avg       0.88      0.87      0.87      7746
weighted avg       0.88      0.87      0.87      7746



Pipeline(steps=[('vectorizer', TfidfVectorizer(max_features=250)),
                ('classifier', SVC(random_state=42))])

#### TF-IDF + Random Forest

In [16]:
pipeline(
    model_type = 'random_forest',
    vectorizer_type = 'tfidf',
    X_train = X_train_text,
    y_train = y_train_text,
    X_test = X_test_text,
    y_test = y_test_text,
    ngram_range = (1, 1)
)

===== Model: RandomForestClassifier with TfidfVectorizer =====
Accuracy: 0.9946
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      3581
           1       0.99      1.00      0.99      4165

    accuracy                           0.99      7746
   macro avg       0.99      0.99      0.99      7746
weighted avg       0.99      0.99      0.99      7746



Pipeline(steps=[('vectorizer', TfidfVectorizer(max_features=250)),
                ('classifier', RandomForestClassifier(random_state=42))])

In [17]:
pipeline(
    model_type = 'random_forest',
    vectorizer_type = 'tfidf',
    X_train = X_train_title,
    y_train = y_train_title,
    X_test = X_test_title,
    y_test = y_test_title,
    ngram_range = (1, 1)
)

===== Model: RandomForestClassifier with TfidfVectorizer =====
Accuracy: 0.8617
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.82      0.85      3581
           1       0.85      0.90      0.88      4165

    accuracy                           0.86      7746
   macro avg       0.86      0.86      0.86      7746
weighted avg       0.86      0.86      0.86      7746



Pipeline(steps=[('vectorizer', TfidfVectorizer(max_features=250)),
                ('classifier', RandomForestClassifier(random_state=42))])

### Neural Network Model

#### CNN

In [14]:
# Tokenizer
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(data["cleaned_title"])

X = tokenizer.texts_to_sequences(data["cleaned_title"])
maxlen = max(len(x) for x in X)
X = pad_sequences(X, maxlen=maxlen)

y = data['label'].values

vocab_size = 5000


In [16]:
# train, validation, test split
X_train_cnn, X_temp_cnn, y_train_cnn, y_temp_cnn = train_test_split(X, y, test_size=0.2, random_state=42)
X_val_cnn, X_test_cnn, y_val_cnn, y_test_cnn = train_test_split(X_temp_cnn, y_temp_cnn, test_size=0.5, random_state=42)

In [19]:
# input shape
inputs = Input(shape=(maxlen,))

# embedding layer
x = Embedding(input_dim=vocab_size, output_dim=64)(inputs)

# convolutional layers
x = Conv1D(64, kernel_size=5, activation='relu')(x)

# Pooling
x = GlobalMaxPooling1D()(x)

# output layer
outputs = Dense(1, activation='sigmoid')(x)

# build the model
model_cnn = Model(inputs=inputs, outputs=outputs)

# compile the model
model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# model summary
model_cnn.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 29)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 29, 64)         │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 25, 64)         │        20,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_1          │ (None, 64)             │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 340,609 (1.30 MB)

 Trainable params: 340,609 (1.30 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

In [21]:
history_cnn = model_cnn.fit(
        X_train_cnn, y_train_cnn,
        validation_data=(X_val_cnn, y_val_cnn),
        epochs=10,
        batch_size=64,
        verbose=1,
        callbacks=[early_stopping]
    )

Epoch 1/10


E0000 00:00:1747917982.849981 119237983 meta_optimizer.cc:967] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


485/485 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.8541 - loss: 0.3534 - val_accuracy: 0.9530 - val_loss: 0.1215
Epoch 2/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - accuracy: 0.9701 - loss: 0.0789 - val_accuracy: 0.9571 - val_loss: 0.1119
Epoch 3/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - accuracy: 0.9851 - loss: 0.0468 - val_accuracy: 0.9556 - val_loss: 0.1285
Epoch 4/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - accuracy: 0.9936 - loss: 0.0241 - val_accuracy: 0.9527 - val_loss: 0.1394
Epoch 5/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - accuracy: 0.9957 - loss: 0.0155 - val_accuracy: 0.9540 - val_loss: 0.1671


In [22]:
# evaluate the model
loss, accuracy = model_cnn.evaluate(X_test_cnn, y_test_cnn, verbose=1)
print(f"Test Accuracy: {accuracy:.4f}")
# make predictions
y_pred_cnn = model_cnn.predict(X_test_cnn)
y_pred_cnn = (y_pred_cnn > 0.5).astype(int)

# classification report
print(f"===== Model: CNN =====")
report = classification_report(y_test_cnn, y_pred_cnn, target_names=['Fake', 'Real'])
print(f"Accuracy: {accuracy:.4f}")
print(f"Classification Report:\n{report}")


122/122 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9538 - loss: 0.1335
Test Accuracy: 0.9494
 70/122 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

E0000 00:00:1747918085.561476 119237983 meta_optimizer.cc:967] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
===== Model: CNN =====
Accuracy: 0.9494
Classification Report:
              precision    recall  f1-score   support

        Fake       0.96      0.93      0.94      1777
        Real       0.94      0.96      0.95      2096

    accuracy                           0.95      3873
   macro avg       0.95      0.95      0.95      3873
weighted avg       0.95      0.95      0.95      3873



#### LSTM model

In [24]:
from gensim.models import Word2Vec

data2 = pd.DataFrame({
    "cleaned_title_tokens": data["cleaned_title"].apply(word_tokenize),
    "label": data["label"]
})

w2v_model = Word2Vec(sentences = data2['cleaned_title_tokens'], vector_size = 50, window = 5, min_count = 2, workers = 10)

In [25]:
tokenizer_lstm = Tokenizer()
tokenizer_lstm.fit_on_texts(data2['cleaned_title_tokens'])
sequences_lstm = tokenizer_lstm.texts_to_sequences(data2['cleaned_title_tokens'])

MAX_LEN = max([len(x) for x in sequences_lstm])
X_seq = pad_sequences(sequences_lstm, maxlen = MAX_LEN)
y = data2["label"].values


In [26]:
import numpy as np

word_index = tokenizer_lstm.word_index
embedding_dim = 50
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))

for word, i in word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]


In [35]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Input layer
input_layer = Input(shape=(MAX_LEN,), name="input")

# Embedding layer
embedding_layer = Embedding(input_dim=len(word_index) + 1,
                            output_dim=embedding_dim,
                            weights=[embedding_matrix],
                            input_length=MAX_LEN,
                            trainable=False)(input_layer)

# LSTM layer
lstm_layer = LSTM(128)(embedding_layer)

# Output layer
output_layer = Dense(1, activation='sigmoid')(lstm_layer)

# Define the model
model_lstm = Model(inputs=input_layer, outputs=output_layer)

# Compile
model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.summary()


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 29)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_5 (Embedding)         │ (None, 29, 50)         │     1,137,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 128)            │        91,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,229,277 (4.69 MB)

 Trainable params: 91,777 (358.50 KB)

 Non-trainable params: 1,137,500 (4.34 MB)

In [36]:
X_train_lstm, X_temp_lstm, y_train_lstm, y_temp_lstm = train_test_split(
    X_seq, y, test_size=0.2, random_state=42, stratify=y
)
X_val_lstm, X_test_lstm, y_val_lstm, y_test_lstm = train_test_split(
    X_temp_lstm, y_temp_lstm, test_size=0.5, random_state=42, stratify=y_temp_lstm
)

In [37]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

In [39]:
# Train the model
model_lstm.fit(X_train_lstm, y_train_lstm, epochs=10, batch_size=64, validation_data=(X_val_lstm, y_val_lstm))


Epoch 1/10


E0000 00:00:1747918882.377482 119237983 meta_optimizer.cc:967] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


485/485 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.8508 - loss: 0.3409 - val_accuracy: 0.8848 - val_loss: 0.2622
Epoch 2/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.8957 - loss: 0.2461 - val_accuracy: 0.9063 - val_loss: 0.2360
Epoch 3/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.9047 - loss: 0.2206 - val_accuracy: 0.9081 - val_loss: 0.2181
Epoch 4/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.9132 - loss: 0.2067 - val_accuracy: 0.9055 - val_loss: 0.2143
Epoch 5/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.9134 - loss: 0.1998 - val_accuracy: 0.9122 - val_loss: 0.2049
Epoch 6/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.9184 - loss: 0.1907 - val_accuracy: 0.9197 - val_loss: 0.1970
Epoch 7/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.9206 - loss: 0.1846 - val_accuracy: 0.9187 - val_loss: 0.1926
Epoch 8/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.9221 - loss: 0.1800 - val_accuracy: 0.91

In [42]:
# evaluate the model
loss, accuracy = model_lstm.evaluate(X_test_lstm, y_test_lstm, verbose=1)
print(f"Test Accuracy: {accuracy:.4f}")
# make predictions
y_pred_lstm = model_lstm.predict(X_test_lstm)
y_pred_lstm = (y_pred_lstm > 0.5).astype(int)

# classification report
print(f"===== Model: LSTM =====")
report = classification_report(y_test_lstm, y_pred_lstm, target_names=['Fake', 'Real'])
print(f"Accuracy: {accuracy:.4f}")
print(f"Classification Report:\n{report}")


122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9268 - loss: 0.1707
Test Accuracy: 0.9254
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
===== Model: LSTM =====
Accuracy: 0.9254
Classification Report:
              precision    recall  f1-score   support

        Fake       0.92      0.91      0.92      1791
        Real       0.93      0.94      0.93      2082

    accuracy                           0.93      3873
   macro avg       0.93      0.92      0.92      3873
weighted avg       0.93      0.93      0.93      3873

